In [1]:
from tensorflow.keras.datasets import mnist
from autoencoder import Autoencoder
import numpy as np
from sklearn.model_selection import train_test_split

2025-07-30 16:44:12.103646: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 16:44:12.115793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753865052.130568   38275 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753865052.135323   38275 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753865052.146078   38275 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
(x_train_full, _), (x_test, _) = mnist.load_data()
x_all = np.concatenate([x_train_full, x_test], axis=0).astype("float32") / 255.0
x_all = x_all[..., np.newaxis]

x_train, x_val = train_test_split(x_all, test_size=0.05, random_state=42)

print("Train shape:", x_train.shape)
print("Validation shape:", x_val.shape)

Train shape: (66500, 28, 28, 1)
Validation shape: (3500, 28, 28, 1)


In [3]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
EPOCHS = 50

In [4]:
input_shape = x_train.shape[1:]
latent_space_dim = 2
decoder_out_filter = 1

In [5]:
autoencoder = Autoencoder(input_shape, latent_space_dim, decoder_out_filter, conv_layers_config=[
    {'filters': 32, 'kernel_size': (3, 3), 'strides': (1, 1)},
    {'filters': 64, 'kernel_size': (3, 3), 'strides': (2, 2)},
    {'filters': 64, 'kernel_size': (3, 3), 'strides': (2, 2)},
    {'filters': 64, 'kernel_size': (3, 3), 'strides': (1, 1)}
])

I0000 00:00:1753865059.222564   38275 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
autoencoder.compile(learning_rate=LEARNING_RATE)

In [7]:
autoencoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_1 (Conv2D)   │ (None, 28, 28, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_1      │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_1      │ (None, 28, 28, 32)     │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_2 (Conv2D)   │ (None, 14, 14, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_2      │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_2      │ (None, 14, 14, 64)     │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_3 (Conv2D)   │ (None, 7, 7, 64)       │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_3      │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_3      │ (None, 7, 7, 64)       │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_4 (Conv2D)   │ (None, 7, 7, 64)       │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_4      │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_4      │ (None, 7, 7, 64)       │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_flatten_layer (Flatten) │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bottleneck_layer        │ (None, 2)              │         6,274 │
│ (Dense)                         │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,618 (389.13 KB)

 Trainable params: 99,170 (387.38 KB)

 Non-trainable params: 448 (1.75 KB)

...

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_1 (Conv2D)   │ (None, 28, 28, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_1      │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_1      │ (None, 28, 28, 32)     │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_2 (Conv2D)   │ (None, 14, 14, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_2      │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_2      │ (None, 14, 14, 64)     │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_3 (Conv2D)   │ (None, 7, 7, 64)       │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_3      │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_3      │ (None, 7, 7, 64)       │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_conv_layer_4 (Conv2D)   │ (None, 7, 7, 64)       │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_batch_norm_layer_4      │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_leaky_relu_layer_4      │ (None, 7, 7, 64)       │             0 │
│ (LeakyReLU)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_flatten_layer (Flatten) │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bottleneck_layer        │ (None, 2)              │         6,274 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_dense_layer (Dense)     │ (None, 3136)           │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_reshape_layer (Reshape) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_conv_transpose_layer_1  │ (None, 7, 7, 64)       │        36,864 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_batch_norm_layer_1      │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 220,962 (863.13 KB)

 Trainable params: 220,130 (859.88 KB)

 Non-trainable params: 832 (3.25 KB)

In [8]:
autoencoder.train(train_config = {
    'x_train': x_train,
    'y_train': x_train, # Autoencoders typically use the same data for input and output
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'validation_data': (x_val, x_val), # Validation data for monitoring
    'shuffle': True
})

Epoch 1/50


I0000 00:00:1753865078.216738   38370 service.cc:152] XLA service 0x73d20401a540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753865078.216785   38370 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-07-30 16:44:38.352357: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753865078.862376   38370 cuda_dnn.cc:529] Loaded cuDNN version 90300


  22/2079 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.2230 - mae: 0.4259 - mse: 0.2230

I0000 00:00:1753865084.254283   38370 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2079/2079 ━━━━━━━━━━━━━━━━━━━━ 28s 9ms/step - loss: 0.0595 - mae: 0.1411 - mse: 0.0595 - val_loss: 0.0455 - val_mae: 0.1047 - val_mse: 0.0455
Epoch 2/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0445 - mae: 0.1055 - mse: 0.0445 - val_loss: 0.0427 - val_mae: 0.1014 - val_mse: 0.0427
Epoch 3/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0419 - mae: 0.0996 - mse: 0.0419 - val_loss: 0.0413 - val_mae: 0.0976 - val_mse: 0.0413
Epoch 4/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0407 - mae: 0.0968 - mse: 0.0407 - val_loss: 0.0402 - val_mae: 0.0959 - val_mse: 0.0402
Epoch 5/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0396 - mae: 0.0945 - mse: 0.0396 - val_loss: 0.0402 - val_mae: 0.0944 - val_mse: 0.0402
Epoch 6/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0390 - mae: 0.0932 - mse: 0.0390 - val_loss: 0.0392 - val_mae: 0.0936 - val_mse: 0.0392
Epoch 7/50
2079/2079 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0384 - mae: 0.0921 - mse: 0.

In [9]:
autoencoder.model_save("autoencoder_model.keras")  # Save the trained model